In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

def guardar_clases_en_txt(class_names, ruta_txt='clases.txt'):
    try:
        with open(ruta_txt, 'w') as f:
            for clase in class_names:
                f.write(f"{clase}\n")
        print(f"Arreglo de clases guardado en: {ruta_txt}")
    except Exception as e:
        print(f"Error al guardar el archivo: {e}")

# Ruta de las imágenes organizadas en carpetas
data_path = "ropa1/"

# Parámetros de imagen
img_size = (64, 64)
batch_size = 32

# Generadores de datos de entrenamiento y validación
train_datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Obtener los nombres de las clases a partir de las carpetas
class_names = list(train_generator.class_indices.keys())
guardar_clases_en_txt(class_names)
print("Class names:", class_names)


# Modelo CNN
input_layer = layers.Input(shape=(64, 64, 3))
x = layers.Conv2D(32, (3, 3), activation='relu')(input_layer)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)

# Capa de salida
output_layer = layers.Dense(train_generator.num_classes, activation='softmax')(x)

# Crear y compilar el modelo
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Entrenamiento del modelo
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

# Evaluación
evaluation_results = model.evaluate(validation_generator)
test_loss, test_acc = evaluation_results[0], evaluation_results[1]

print('\nTest loss:', test_loss)
print('Test accuracy:', test_acc)
print('Class names:', class_names)  # Muestra los nombres de las clases según las carpetas


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def predict_image(model, img_path, class_names, target_size=(64, 64)):
    # Cargar la imagen y redimensionarla
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img)
    
    # Normalizar la imagen (igual que durante el entrenamiento)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Añadir una dimensión para el batch
    
    # Realizar la predicción
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class_name = class_names[predicted_class_index]
    
    # Mostrar el resultado
    print(f"Predicted class: {predicted_class_name}")
    return predicted_class_name

# Prueba de la función con una imagen de ejemplo
img_path = 'ropa/hoodie.jpg'
predicted_class = predict_image(model, img_path, class_names)


In [ ]:
import tensorflow as tf

# Suponiendo que ya tienes un modelo cargado o creado
# model = ... (cargar o crear tu modelo aquí)

# Guardar el modelo en formato Keras
model.save("modelo/modelo.keras")

# Cargar el modelo desde el archivo Keras
model = tf.keras.models.load_model("modelo/modelo.keras")

# Guardar el modelo en formato SavedModel
model.export("modelo/saved_model")

print("Modelo guardado en formato SavedModel en 'modelo/saved_model'")
